<a href="https://colab.research.google.com/github/sanaeaba/Recommende_system_coursera/blob/master/NCF_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install recommenders
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
from sklearn.model_selection import train_test_split


import recommenders
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Pandas version: 1.3.5
Tensorflow version: 2.8.2


In [ ]:
# top k items to recommend
TOP_K = 30

# Model parameters
EPOCHS = 20
BATCH_SIZE = 256

SEED = 42


In [ ]:
# loading coursera data
df = pd.read_csv("coursera_for_ncf.csv")
df

,Unnamed: 0,itemID,userID,rating,timestamp
0,0,289,8507,2,881250949
1,1,289,15643,2,891717742
2,2,289,3474,2,878887116
3,3,289,7221,3,880606923
4,4,289,8776,3,886397596
...,...,...,...,...,...
20117,20117,210,1060,4,877196173
20118,20118,210,4906,4,882912418
20119,20119,210,838,4,880132055
20120,20120,210,695,4,877844489


In [ ]:
df= df.drop(columns=['Unnamed: 0'])

In [ ]:
 df.columns

Index(['itemID', 'userID', 'rating', 'timestamp'], dtype='object')

In [ ]:
# chronological spilt on every user

train, test = python_chrono_split(df, 0.75)

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [ ]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [ ]:
# preparing the data
# This function by default takes sampling on unobserved entries to account for negative feedback 
# It only takes rows which atleast have user id, item id and rating
# This data contains rating and hence is the textbook definition of explicit data

data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file ./test_full.csv ...
100%|██████████| 318/318 [00:02<00:00, 124.38it/s]
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


In [ ]:
# n_factors (int): Dimension of latent space.
# layer_sizes (list): Number of layers for MLP.

model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="neumf",
    n_factors=2,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=1,
    seed=SEED
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
# training the model

with Timer() as train_time:
    history = model.fit(data)
        
print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 1 [379.15s]: train_loss = 0.568751 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 2 [379.38s]: train_loss = 0.390917 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 3 [366.07s]: train_loss = 0.379181 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 4 [377.27s]: train_loss = 0.360477 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 5 [381.61s]: train_loss = 0.336131 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 6 [386.81s]: train_loss = 0.308645 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 7 [367.56s]: train_loss = 0.282113 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 8 [365.38s]: train_loss = 0.256884 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 9 [358.02s]: train_loss = 0.237499 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [363.14s]: train_loss = 0.224593 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 11 [357.89s]: train_loss = 0.214405 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 12

Took 7348.1927 seconds for training.


In [ ]:
for index, row in test.iterrows():
  predict = model.predict(row.userID, row.itemID)

In [ ]:
predict

0.6358354091644287

In [ ]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# saving the predictions in a dataframe
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,226,270,0.001239
1,234,203,0.160049
2,262,238,0.895816
3,326,7,0.062870
4,363,155,0.024395


# **Evaluation**

In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 23.6910 seconds for prediction.


In [ ]:
MAE = mae(test, all_predictions, col_prediction='prediction')
RMSE = rmse(test, all_predictions, col_prediction='prediction')


print("MAE:\t%f" % MAE,
      "RMSE:\t%f" % RMSE, sep='\n')

MAE:	4.156625
RMSE:	4.305089
